In [10]:
import numpy as np
import os
import sys
import glob
import pandas as pd
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics.metrics import ModelPerformance, Metrics

model_name = 'ProjNetMerged4'
filepaths = glob.glob('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/'
    f'models/predict_loco/{model_name}/csv/*')

filepaths.sort()

filepath = filepaths[-1]

print(filepath)
#mp = ModelPerformance.load(filepath=filepath)
#mp['Model'] = model_name

df_loco = pd.read_csv(filepath)

df_loco = df_loco.drop(columns = ['massive_filename'])


levels = (0, 16, 17, 7, 8, 18, 19, 15)
contacts = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
contact_to_levels = {
    0 : 0,
    16 : 1,
    17 : 2,
    7 : 3,
    8 : 0,
    18 : 1,
    19 : 2,
    15 : 3}
contact_to_hemisphere_lvl = {
    0 : 'right',
    16 : 'right',
    17 : 'right',
    7 : 'right',
    8 : 'left',
    18 : 'left',
    19 : 'left',
    15 : 'left'}
contact_to_hemisphere_vercise = {
    0 : 'right', # R0R
    1 : 'right', # R1_0R
    2 : 'right', # R1_1R
    3 : 'right', # R1_2R
    4 : 'right', # R2_0R
    5 : 'right', # R2_1R
    6 : 'right', # R2_2R
    7 : 'right', # R3R
    8 : 'left',  # R0L
    9 : 'left',  # R1_0L
    10 : 'left', # R1_1L
    11 : 'left', # R1_2L
    12 : 'left', # R2_0L
    13 : 'left', # R2_1L
    14 : 'left', # R2_2L
    15 : 'left', # R3L
    16 : 'right',# R1R
    17 : 'right',# R2R
    18 : 'left', # R1L
    19 : 'left'  # R2L
    }

df_loco['hemisphere'] = df_loco['verciseID'].map(contact_to_hemisphere_vercise)

model_name = 'ProjNetMerged3'
filepaths = glob.glob('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/'
    f'models/predict_lopo/{model_name}/csv/*')

filepaths.sort()

filepath = filepaths[-1]

print(filepath)
#mp = ModelPerformance.load(filepath=filepath)
#mp['Model'] = model_name

df_lopo = pd.read_csv(filepath)

df_lopo = df_lopo.drop(columns = ['massive_filename'])
#df = df[['patientID', 'hemisphere', 'contactID', 'amplitude', 'label', 'prediction']]
df_lopo['hemisphere'] = df_lopo['verciseID'].map(contact_to_hemisphere_vercise)

/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/models/predict_loco/ProjNetMerged4/csv/23.09.13.17:08:28_es.csv
/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/models/predict_lopo/ProjNetMerged3/csv/23.09.11.16:18:27_es.csv


In [11]:
a = df_loco.patientID == 217
b = df_loco.contactID == 0
c = df_loco.added_voxels > 0

df_loco[a & b & c].sort_values(by=['amplitude'])[['centerID', 'patientID', 'verciseID', 'amplitude', 'step_interp_score']]

,centerID,patientID,verciseID,amplitude,step_interp_score
346425,Cologne,217.0,0,0.5,0.0
474393,Cologne,217.0,0,0.5,0.0
322001,Cologne,217.0,0,0.5,0.0
323200,Cologne,217.0,0,0.5,0.0
524506,Cologne,217.0,0,0.5,0.0
...,...,...,...,...,...
577929,Cologne,217.0,0,4.9,0.0
531880,Cologne,217.0,0,4.9,0.0
582422,Cologne,217.0,0,4.9,0.0
504488,Cologne,217.0,0,4.9,0.0


In [12]:
import pandas as pd

# Assuming 'df_loco' is your DataFrame
# Create an empty column for 'abs_interp_error'
df_loco['abs_interp_error'] = None

# Step 1: Identify reference rows where 'noisy' == 0
reference_df = df_loco[df_loco['noisy'] == 0]

# Step 2: Calculate absolute difference for each group
for name, group in df_loco.groupby(['centerID', 'patientID', 'verciseID', 'amplitude']):
    reference_score = reference_df.loc[
        (reference_df['centerID'] == name[0]) & 
        (reference_df['patientID'] == name[1]) & 
        (reference_df['verciseID'] == name[2]) & 
        (reference_df['amplitude'] == name[3]), 
        'step_interp_score'
    ].values[0]  # Assuming there is one reference row per group
    
    abs_error = abs(group['step_interp_score'] - reference_score)
    
    # Step 3: Store these differences back into 'abs_interp_error'
    df_loco.loc[group.index, 'abs_interp_error'] = abs_error

# Step 4: Calculate and display average 'abs_interp_error' for each 'centerID'
result = df_loco.groupby('centerID')['abs_interp_error'].mean()
print(result)


centerID
Bern       0.000189
Cologne    0.001675
Name: abs_interp_error, dtype: object


In [13]:
result = df_loco.groupby('centerID')['abs_interp_error'].sum()
print(result)

centerID
Bern        56.0
Cologne    495.0
Name: abs_interp_error, dtype: object


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display


df = df_loco

# Grouping the dataframe
grouped_df = df.groupby(['patientID', 'contactID'])

# Extracting the unique group indices
group_indices = grouped_df.groups.keys()
group_indices = sorted(group_indices)

# Extract unique patient IDs and contact IDs for each patient
unique_patient_ids = sorted(df['patientID'].unique())
patient_to_contacts = df.groupby('patientID')['contactID'].unique().to_dict()

# Create widgets
patient_id_widget = widgets.Dropdown(
    options=unique_patient_ids,
    description='Patient ID:'
)

contact_id_widget = widgets.Dropdown(
    options=patient_to_contacts.get(unique_patient_ids[0], []),
    description='Contact ID:'
)

score_type_widget = widgets.Dropdown(
    options=['lin_interp_score', 'step_interp_score'],
    value='lin_interp_score',
    description='Score Type:'
)

# Update function to update contacts based on selected patient
def update_contact_id_widget(*args):
    selected_patient = patient_id_widget.value
    contact_id_widget.options = patient_to_contacts.get(selected_patient, [])

# Attach update function to patient ID widget
patient_id_widget.observe(update_contact_id_widget, 'value')


def plot_group(patient_id, contact_id, score_type):
    # Getting the corresponding group
    group = (patient_id, contact_id)
    data = grouped_df.get_group(group).sort_values(by='amplitude')

    # Create a color palette dictionary
    palette = {
        True: 'red',  # mapping
        False: 'blue' if score_type == 'lin_interp_score' else 'green'  # score_type
    }

    plt.figure(figsize=(10, 6))

    sns.scatterplot(data=data, x='amplitude', y=score_type, hue='mapping', palette=palette, s=10, alpha=.3)

    # Handle noisy points specifically
    noisy_data = data[data['noisy'] == True]
    plt.scatter(noisy_data['amplitude'], noisy_data[score_type], color='darkgreen', s=10, alpha=.3)

    plt.xlabel('Amplitude')
    plt.ylabel(score_type.capitalize())
    plt.title(f'Amplitude vs {score_type.capitalize()} (Group: {group})')

    plt.ylim(-0.1, 1.1)
    plt.xlim(0, 8.5)
    
    plt.grid(True)
    plt.legend(title='Mapping')
    plt.show()

# Create interactive widget
interactive_plot = widgets.interactive(plot_group, patient_id=patient_id_widget, contact_id=contact_id_widget, score_type=score_type_widget)

# Display widgets
display(patient_id_widget, contact_id_widget, score_type_widget, interactive_plot)


Dropdown(description='Patient ID:', options=(1.0, 3.0, 6.0, 8.0, 9.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0…

Dropdown(description='Contact ID:', options=(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.…

Dropdown(description='Score Type:', options=('lin_interp_score', 'step_interp_score'), value='lin_interp_score…

interactive(children=(Dropdown(description='Patient ID:', options=(1.0, 3.0, 6.0, 8.0, 9.0, 11.0, 12.0, 13.0, …